## Configuración y Herramientas
Importamos `requests` para navegar por internet, `tiktoken` para tokenizar, y `urllib` para limpiar los nombres de las webs.

### Ejercicio 1: Analizando Libros Clásicos (Proyecto Gutenberg)

Vamos a descargar 10 libros clásicos y calcular su **Ratio de Compresión**.
La fórmula es: `Caracteres / Tokens`.
* Un número alto (ej: 4.0) significa buena compresión (el modelo lee rápido).
* Un número bajo (ej: 1.5) significa mala compresión.

*Nota: La URL se construye dinámicamente usando el código del libro.*

In [2]:
# for getting text data off the web
import requests
import re
from urllib.parse import urlparse
# strings
import string

# !pip install tiktoken # Descomentar si es necesario
import tiktoken

# GPT-4's tokenizer
tokenizer = tiktoken.get_encoding('cl100k_base')

### Ejercicio 1: Analizando Libros Clásicos (Proyecto Gutenberg)

Vamos a descargar 10 libros clásicos y calcular su **Ratio de Compresión**.
La fórmula es: `Caracteres / Tokens`.
* Un número alto (ej: 4.0) significa buena compresión (el modelo lee rápido).
* Un número bajo (ej: 1.5) significa mala compresión.

*Nota: La URL se construye dinámicamente usando el código del libro.*

In [3]:
# Exercise 1: The books

# all books have the same url format;
# they are unique by numerical code
baseurl = 'https://www.gutenberg.org/cache/epub/'

bookurls = [
    # code       title
    ['84',    'Frankenstein'    ],
    ['64317', 'GreatGatsby'     ],
    ['11',    'AliceWonderland' ],
    ['1513',  'RomeoJuliet'     ],
    ['76',    'HuckFinn'        ],
    ['219',   'HeartDarkness'   ],
    ['2591',  'GrimmsTales'     ],
    ['2148',  'EdgarAllenPoe'   ],
    ['36',    'WarOfTheWorlds'  ],
    ['829',   'GulliversTravels']
]


print('  Book title     |  Chars  |  Tokens | Compression')
print('-'*50)

for code, title in bookurls:

  # get the text
  # Construimos la URL completa (ej: .../84/pg84.txt)
  fullurl = f'{baseurl}{code}/pg{code}.txt'

  try:
      response = requests.get(fullurl)
      response.encoding = 'utf-8' # Asegurar codificación correcta
      text = response.text
  except:
      text = "" # Si falla, texto vacío

  num_chars = len(text)

  # tokenize
  if num_chars > 0:
      tokens = tokenizer.encode(text)
      num_tokens = len(tokens)
  else:
      num_tokens = 1 # Evitar división por cero

  # compression ratio (Caracteres por Token)
  compress = num_chars / num_tokens

  print(f'{title:16} | {num_chars:>7,d} | {num_tokens:>7,d} |  {compress:>3.2f} chars/token')

  Book title     |  Chars  |  Tokens | Compression
--------------------------------------------------
Frankenstein     | 446,544 | 102,419 |  4.36 chars/token
GreatGatsby      | 296,858 |  70,343 |  4.22 chars/token
AliceWonderland  | 167,674 |  41,457 |  4.04 chars/token
RomeoJuliet      | 167,429 |  43,761 |  3.83 chars/token
HuckFinn         | 602,714 | 159,125 |  3.79 chars/token
HeartDarkness    | 232,885 |  56,483 |  4.12 chars/token
GrimmsTales      | 549,736 | 137,252 |  4.01 chars/token
EdgarAllenPoe    | 632,136 | 144,315 |  4.38 chars/token
WarOfTheWorlds   | 363,420 |  84,580 |  4.30 chars/token
GulliversTravels | 611,742 | 143,560 |  4.26 chars/token


### Ejercicio 2: Analizando Sitios Web (Código HTML)

Ahora haremos lo mismo con sitios web.
**Hipótesis:** Como las webs tienen mucho código HTML (`<div>`, `<script>`, `href=...`), esperamos que la compresión sea **peor** (más baja) que en los libros, porque el código tiene muchos símbolos que rompen los tokens.

In [4]:
# Exercise 2: Repeat with websites

weburls = [
    'http://python.org/',
    'https://pytorch.org/',
    'https://en.wikipedia.org/wiki/List_of_English_words_containing_Q_not_followed_by_U',
    'https://sudoku.com/',
    'https://reddit.com/',
    'https://visiteurope.com/en/',
    'https://sincxpress.com/',
    'https://openai.com/',
    'https://theuselessweb.com/',
    'https://maps.google.com/',
    'https://pigeonsarentreal.co.uk/',
]


print('    Website        |  Chars  |  Tokens | Compression')
print('-'*53)

for url in weburls:

  # get the text
  try:
      # Usamos headers para simular un navegador real y evitar bloqueos (403 Forbidden)
      headers = {'User-Agent': 'Mozilla/5.0'}
      text = requests.get(url, headers=headers).text
  except:
      text = "Error"

  num_chars = len(text)

  # tokenize
  if num_chars > 0:
      tokens = tokenizer.encode(text)
      num_tokens = len(tokens)
  else:
      num_tokens = 1

  # compression ratio
  compress = num_chars / num_tokens

  # Usamos urlparse para obtener solo el nombre limpio del sitio (hostname)
  hostname = urlparse(url).hostname
  if hostname is None: hostname = "unknown"

  print(f'{hostname[:18]:18} | {num_chars:>7,d} | {num_tokens:>7,d} |  {compress:>3.2f} chars/token')

    Website        |  Chars  |  Tokens | Compression
-----------------------------------------------------
python.org         |  48,288 |  12,236 |  3.95 chars/token
pytorch.org        | 184,926 |  56,827 |  3.25 chars/token
en.wikipedia.org   | 158,792 |  50,812 |  3.13 chars/token
sudoku.com         | 140,695 |  51,330 |  2.74 chars/token
reddit.com         |   1,522 |     395 |  3.85 chars/token
visiteurope.com    | 405,838 | 155,130 |  2.62 chars/token
sincxpress.com     |  25,580 |   6,843 |  3.74 chars/token
openai.com         |  11,639 |   6,424 |  1.81 chars/token
theuselessweb.com  |   4,756 |   1,329 |  3.58 chars/token
maps.google.com    | 214,606 | 109,076 |  1.97 chars/token
pigeonsarentreal.c |   7,300 |   2,051 |  3.56 chars/token


### Ejercicio 3: Analizando la Librería `string`

Python tiene una librería llamada `string` que contiene constantes como "todas las letras minúsculas" o "todos los signos de puntuación".
Vamos a ver qué tan eficientes son estos grupos de caracteres.
* Los dígitos suelen ser eficientes.
* La puntuación suele ser ineficiente (1 caracter = 1 token).

In [5]:
# Exercise 3: Using the 'string' library

# e.g., inspect available attributes
# string.__dict__['__doc__']
# string.ascii_lowercase

print('  Attribute     |  Chars  |  Tokens | Compression')
print('-'*50)

for k,v in string.__dict__.items():
  # Check if the attribute is a string (ignoring functions or classes)
  if isinstance(v, str):

    # get the text
    text = v
    num_chars = len(text)

    # tokenize
    tokens = tokenizer.encode(text)
    num_tokens = len(tokens)

    # compression ratio
    if num_tokens > 0:
        compress = num_chars / num_tokens
    else:
        compress = 0

    # print the results (Omitimos atributos vacíos o muy cortos para limpiar la salida)
    if num_chars > 5:
        print(f'{k:15} | {num_chars:>7,d} | {num_tokens:>7,d} |  {compress:>3.2f} chars/token')

  Attribute     |  Chars  |  Tokens | Compression
--------------------------------------------------
__name__        |       6 |       1 |  6.00 chars/token
__doc__         |     622 |     109 |  5.71 chars/token
__file__        |      29 |       8 |  3.62 chars/token
__cached__      |      54 |      17 |  3.18 chars/token
whitespace      |       6 |       4 |  1.50 chars/token
ascii_lowercase |      26 |       1 |  26.00 chars/token
ascii_uppercase |      26 |       1 |  26.00 chars/token
ascii_letters   |      52 |       2 |  26.00 chars/token
digits          |      10 |       4 |  2.50 chars/token
hexdigits       |      22 |       7 |  3.14 chars/token
octdigits       |       8 |       3 |  2.67 chars/token
punctuation     |      32 |      21 |  1.52 chars/token
printable       |     100 |      31 |  3.23 chars/token


# Explicación: Métricas de Compresión y Eficiencia

En esta sección estamos calculando el **Ratio de Compresión** (Efficiency Score).
La fórmula es simple:
$$\text{Eficiencia} = \frac{\text{Caracteres totales}}{\text{Tokens totales}}$$

### ¿Por qué nos importa esto?

OpenAI (y la mayoría de las APIs de IA) te cobran por **token**, no por palabra ni por carácter.
* **Alta Eficiencia (ej: 4.0):** Significa que cada token transporta mucha información (muchas letras). Es **barato**.
* **Baja Eficiencia (ej: 1.2):** Significa que necesitas muchos tokens para decir poco. Es **caro**.

### Análisis de los Resultados

#### 1. Libros (Lenguaje Natural)
Verás que los libros clásicos (*Frankenstein*, *Alice in Wonderland*) tienen una eficiencia alta (alrededor de **3.5 a 4.5** caracteres por token).
* **Razón:** El algoritmo BPE fue entrenado con mucho texto en inglés. Sabe comprimir palabras comunes como "the", "ing", "tion" en un solo token.

#### 2. Sitios Web (Código HTML y Ruido)
Verás que la eficiencia cae drásticamente (alrededor de **1.5 a 2.5**).
* **Razón:** El código HTML está lleno de símbolos que rompen los tokens (`<`, `>`, `/`, `=`, `"`).
* **Ejemplo:** La palabra `class` es un token. Pero en HTML suele aparecer como `class="estilo"`. Las comillas y el igual obligan al tokenizador a separar todo en pedazos pequeños.

#### 3. Librería String (Categorías Puras)
Aquí vemos los extremos:
* **Dígitos:** Suelen ser eficientes o neutros.
* **Puntuación (`punctuation`):** Es lo peor. Generalmente **1 carácter = 1 token**.

### 💡 Lección para tu Negocio de Automatización
Si vas a automatizar la lectura de webs o bases de datos:
**Limpia el código antes de enviarlo a GPT.**
Si envías HTML crudo (`<div><h1>Hola</h1></div>`), estás pagando por tokens de etiquetas que no aportan significado. Si limpias el texto ("Hola"), ahorras dinero y el modelo entiende mejor.